# Начало работы

Руководство [здесь](https://metanit.com/python/tkinter/)

In [ ]:
!pip install tk
# % для VS Code

In [2]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog

In [ ]:
# установки библиотек
!pip install wordcloud
!pip install pillow
!pip install pymorphy3 
!pip install torch
!pip install transformers

# % для VS Code

In [ ]:
# для очистки (препроцессинга) текста...
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
#nltk.download('punkt_tab')
from nltk.corpus import stopwords
#nltk.download('stopwords')
ru_stop_words = stopwords.words('russian')

# pymorphy
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

#таблички
import pandas as pd

# для рисования облака слов
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# для подсчета любых частот (часто встречающиеся слова, n-граммы и т.д.)
from collections import Counter

# картинки
from PIL import ImageTk, Image  

In [ ]:
# сентимент-анализ

import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
tokenizer_sentiment = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model_sentiment = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

In [6]:
tk._test()

In [7]:
window = Tk()
window.title("Первая программа")
window.mainloop() # вызывает окошко

In [8]:
window = Tk()
window.title("Первая программа")
window.geometry('500x400') # размер окошечка
window.mainloop()

In [9]:
window = Tk()
window.title("Первая программа")
window.geometry('500x400') # размер окошечка

lbl = Label(window, text="Привет, мир", font=("Arial", 30))  
lbl.pack() 
window.mainloop()

In [10]:
window = Tk()
window.title("Первая программа")
window.geometry('500x400') # размер окошечка

btn = Button(window, 
             text="Просто кнопка") # текст на кнопке
btn.pack() # расположить кнопку на главном окне

window.mainloop()

In [ ]:
def Hello(event):
    print("Здравствуй, мир!")

window = Tk()
window.title("Первая программа")
window.geometry('500x400') # размер окошечка

btn = Button(window,                  
             text="Приветствие",     #надпись на кнопке
             width=30, height=5,     #ширина и высота
             bg="white", fg="black") #цвет фона и надписи
btn.bind("<Button-1>", Hello)        #при нажатии ЛКМ на кнопку вызывается функция Hello
btn.pack()                           #расположить кнопку на главном окне

window.mainloop()

Добавим для кнопки какое-нибудь действие...

In [12]:
def Hello(event):
    messagebox.showinfo(title="Приветствие", message = "Здравствуй, мир!") #Добавляем событие для функции

window = Tk()
window.title("Первая программа")
window.geometry('500x400') # размер окошечка

btn = Button(window,                  
             text="Приветствие",     #надпись на кнопке
             width=30, height=5,     #ширина и высота
             bg="white", fg="black", #цвет фона и надписи
             font='Arial',           #шрифт надписи
             command=Hello)          #gозволяет запустить событие с функцией при нажатии на кнопку.

btn.bind("<Button-1>", Hello)        #при нажатии ЛКМ на кнопку вызывается функция Hello
btn.pack()                           #расположить кнопку на главном окне

window.mainloop()

# Пишем NLP-программу

In [13]:
def get_ru_lemm(text):
  text = text.lower()
  text_tokens = word_tokenize(text)
  clean_tokens = []
  for i in text_tokens:
    if i[0].isalpha() and i not in ru_stop_words:
      clean_tokens.append(i)
  ru_lemmatized = []
  for word in clean_tokens:
      ru_lemmatized.append(morph.parse(word)[0].normal_form)
  return ru_lemmatized

def n_grams(lemm):
  freq_bigramms = Counter(nltk.bigrams(lemm)).most_common(10)
  res_bigramms = []
  for i, j in freq_bigramms:
    res_bigramms.append(''.join((' '.join(i)) + ': ' + str(j)))
  freq_trigramms = Counter(nltk.trigrams(lemm)).most_common(5)
  res_trigramms = []
  for i, j in freq_trigramms:
    res_trigramms.append(''.join((' '.join(i)) + ': ' + str(j)))
  return res_bigramms, res_trigramms

def word_cloud(lemm):
  wordcloud = WordCloud(width = 1000,
                        height = 750,
                        background_color='white',
                        colormap='tab10').generate(', '.join(lemm))
  plt.figure(figsize=(20, 15)) # Устанавливаем размер картинки
  plt.imshow(wordcloud) # Что изображаем
  plt.axis("off") # Без подписей на осях
  plt.savefig('wordcoud')

def predict_bert(lemm):
  positive = []
  negative = []
  for i in lemm:
    inputs = tokenizer_sentiment(i, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model_sentiment(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.topk(predicted, k=1)
    if predicted.indices.detach().numpy()[0] == 1:
      positive.append((i, float(predicted.values.detach().numpy()[0][0])))
    elif predicted.indices.detach().numpy()[0] == 2:
      negative.append((i, float(predicted.values.detach().numpy()[0][0])))
  positive.sort(key=lambda x: x[1], reverse=True)
  negative.sort(key=lambda x: x[1], reverse=True)
  most_positive_words = []
  for w, v in positive:
    if w not in most_positive_words:
      most_positive_words.append(w)
    if len(most_positive_words) < 5:
      continue
    else:
      break
  most_negative_words = []
  for w, v in negative:
    if w not in most_negative_words:
      most_negative_words.append(w)
    if len(most_negative_words) < 5:
      continue
    else:
      break
  return most_positive_words, most_negative_words

### Добаляем ей кнопку для загрузки текста пользователем

In [14]:
def openFile():
    tf = filedialog.askopenfilename(
        initialdir="home/", # "C:/Users/.../Desktop/"
        title="Open Text file", 
        filetypes=(("Text Files", "*.txt"),)
        )
    pathh.insert(END, tf)
    tf = open(tf, 'r')
    data = tf.read()
    txtarea.insert(END, data)
    tf.close()

window = Tk()
window.title("Мини-анализ текста")
window.geometry('800x600') 

txtarea = Text(window, width=40, height=20)
txtarea.pack(pady=20)

pathh = Entry(window)
pathh.pack(side=LEFT, expand=True, fill=X, padx=20)

Button(
    window, 
    text="Открыть файл", 
    command=openFile
    ).pack(side=RIGHT, expand=True, fill=X, padx=20)

window.mainloop()

### Финальная версия

In [ ]:
def open_img():
    img = Image.open("wordcoud.png")
    img = img.resize((500, 400), Image.Resampling.LANCZOS)
    img = ImageTk.PhotoImage(img)
    panel = Label(window, image=img)
    panel.image = img
    panel.grid(column=0, row=3)

def openFile():
    tf = filedialog.askopenfilename(
        initialdir="home/", # "C:/Users/MainFrame/Desktop/"
        title="Open Text file", 
        filetypes=(("Text Files", "*.txt"),)
        )
    pathh.insert(END, tf)
    tf = open(tf, 'r')
    data = tf.read()
    
    lemm = get_ru_lemm(data)
    txtarea.insert(END, ', '.join(lemm))
    
    ngram = n_grams(lemm)
    txtarea2.insert(END, '\n'.join(ngram[0]) + '\n\n' +
                   '\n'.join(ngram[1]))
    
    sent = predict_bert(lemm)
    txtarea3.insert(END, 'Позитивно окрашенные слова: \n' + ', '.join(sent[0]) + '\n\n' +
                   'Негативно окрашенные слова: \n' + ', '.join(sent[1]))
    
    word_cloud(lemm)
    
    tf.close()

window = Tk()
window.title("Мини-анализ текста")
window.geometry('1150x1000') 

txtarea = Text(window, width=160, height=20)
txtarea.grid(column=0, row=0, padx=5, pady=5)
                   
txtarea2 = Text(window, width=75, height=20)
txtarea2.grid(column=0, row=1, sticky='w', padx=5, pady=5)

txtarea3 = Text(window, width=75, height=20)
txtarea3.grid(column=0, row=1, sticky='e', padx=5, pady=5)

pathh = Entry(window, width=100)
pathh.grid(column=0, row=2, padx=5, pady=5)

open_button = Button(
    window, 
    text="Загрузите файл", 
    command=openFile,
    width=30, height=5
    )
open_button.grid(column=0, row=3, sticky='w', padx=5, pady=5)

img_btn = Button(window, 
                 text='Облако слов', 
                 command=open_img,
                 width=30, height=5)
img_btn.grid(column=0, row=3, sticky='e')

window.mainloop()